## Plotting library

This is not going to be needed much so it can be skipped.

In [1]:
using Plots
gr()

Plots.GRBackend()

## PDMP libraries + JLD and co

Takes 10 seconds to load in Julia 0.6, this is due to ApproxFun... 

In [2]:
print("loading PDMP... "); ta = time()
using PDMP
println("[done in $(round(time()-ta,1))s]")

print("loading other packages... "); ta = time()
using JLD
println("[done in $(round(time()-ta,1))s]")

cprint(s, b)   = b ? print(s)   : nothing
cprintln(s, b) = b ? println(s) : nothing
;

loading PDMP... [done in 9.4s]
loading other packages... [done in 1.5s]


## Loading & prepping of data

Note: the scaling (or absence thereof) of the data, changes the scale of the RMSE. One has to be careful about that before comparing with "benchmark data".

In [3]:
verb = true

cprint("reading and preparing data... ", verb) ; ta = time()

# This is the Movielens 1M dataset
rows      = vec(readdlm("data/rows.csv",  Int))
cols      = vec(readdlm("data/cols.csv",  Int))
raw_rates = vec(readdlm("data/rates.csv", Float64))

raw_rates_c = raw_rates-mean(raw_rates)

# centre and scale the rates
range     = maximum(raw_rates)-minimum(raw_rates)
nrm_rates = (raw_rates_c)/range

# scaling as per Salakhutdinov & Mni
pmf_rates = (raw_rates - minimum(raw_rates))/range

cprintln("[done in $(round(time()-ta,1))s]", verb)
;

reading and preparing data... [done in 1.9s]


### Picking one to go with

In [4]:
#rates = pmf_rates
rates = raw_rates_c
;

## Splitting train & test

90% for training, remaining for test.

In [5]:
srand(555)

nfull      = length(rates)
ntrain     = round(Int,0.90*nfull)
mask       = randperm(nfull)
train_mask = mask[1:ntrain]
test_mask  = mask[(ntrain+1):end]

ntest = length(test_mask)
;

## Computation of base sigmas

* $\sigma_R = 0.5$ (orig paper)
* $\sigma_U, \sigma_V$ set as per https://pymc-devs.github.io/pymc3/notebooks/pmf-pymc.html

In [6]:
rs = rates[train_mask]

nU = maximum(rows)
nV = maximum(cols)

cU,sU,s2U = zeros(nU), zeros(nU), zeros(nU)
cV,sV,s2V = zeros(nV), zeros(nV), zeros(nV)

for (k,rk) in enumerate(rs)
    cU[rows[k]]  += 1
    sU[rows[k]]  += rk
    s2U[rows[k]] += rk^2
    cV[cols[k]]  += 1
    sV[cols[k]]  += rk
    s2V[cols[k]] += rk^2
end
vU = (s2U ./ cU) - (sU ./ cU).^2
vV = (s2V ./ cV) - (sV ./ cV).^2

vU[vU.<1e-10]=0.0
vV[vV.<1e-10]=0.0


base_sigma_r = 0.5*4 # Salakhutdinov & Mni (scaled)
# https://pymc-devs.github.io/pymc3/notebooks/pmf-pymc.html
base_sigma_u = mean(sqrt.(vU[.~isnan.(vU)]))
base_sigma_v = mean(sqrt.(vV[.~isnan.(vV)]))

println(base_sigma_r)
println(base_sigma_u)
println(base_sigma_v)

2.0
1.1050959450267932
1.044191506613901


### Studying the 0 vector

In [7]:
println("train vs 0vec: ", 
    round(sqrt(sum(rates[train_mask].^2)/ntrain),4))
println("test  vs 0vec: ",
    round(sqrt(sum(rates[test_mask].^2)/ntest),4))

train vs 0vec: 1.1174
test  vs 0vec: 1.1145


In [43]:
include("pmf_rmse.jl")
;

## SVD

In [9]:
d = 10

spmat = sparse(
            rows[train_mask],
            cols[train_mask],
            rates[train_mask]
        )
S = svds(spmat, nsv=d)[1]

sS = sqrt.(S.S)

xSVD  = [vec(diagm(sS) * S.U'); vec(diagm(sS) * S.Vt)]
;

In [10]:
# training error
println("training rmse (SVD): ",
    pmf_rmse(rows[train_mask], cols[train_mask], rates[train_mask], 
                nU, nV, d, xSVD))
println("testing rmse (SVD): ",
    pmf_rmse(rows[test_mask], cols[test_mask], rates[test_mask],
                nU, nV, d, xSVD))

training rmse (SVD): 0.9886040071534168
testing rmse (SVD): 1.0027202112661955


## LBPS runs

In [11]:
include("pmf_lbps.jl")

pmf_lbps (generic function with 2 methods)

In [12]:
data = Dict(
    "ROWS"  => rows[train_mask],
    "COLS"  => cols[train_mask],
    "RATES" => rates[train_mask]
);

In [34]:
d  = 10
sU = base_sigma_u
sV = base_sigma_v
sR = base_sigma_r
lr = 0.01
mn = 1000
mt = Inf

en = "d$d-sU$(round(sU,2))-sV$(round(sV,2))-sR$sR-lr$lr-mn$mn-mt$mt"

# draw x0 from spherical priors
x0 = xSVD

lbpsparams = Dict(
    "EXPNAME"    => en, # name of the experiment
    "LATENT_D"   => d,  # dimension of latent space
    "SIGMA_U"    => sU, #
    "SIGMA_V"    => sV, #
    "SIGMA_R"    => sR, #
    "X0"         => x0, #
    "LAMBDAREF"  => lr, # refreshment rate
    "MAXNEVENTS" => mn, # maximum number of events to generate
    "MAXT"       => mt, # maximum time
)
ta      = time()
results = pmf_lbps(data, lbpsparams)
simtime = round(time()-ta, 2)

# ------------------------------------

pm  = pathmean(results["ALL_EVLIST"])
pmu = pm[1:nU]
pmv = pm[nU+1:end]

xx = similar(x0)
for i in 1:length(pm)
    xx[((i-1)*d+1):(i*d)] = pm[i]
end

;

Starting LBPS...preparing the graph... [done in 4.8s]
initialising the simulation... [done in 0.0s]
starting the simulation... 


Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] DataStructures.PriorityQueue(::Type{Int64}, ::Type{Float64}) at ./deprecated.jl:57
 [3] ls_init(::PDMP.LocalSimulation) at /Users/tlienart/.julia/v0.6/PDMP/src/local/simulate.jl:129
 [4] simulate(::PDMP.LocalSimulation) at /Users/tlienart/.julia/v0.6/PDMP/src/local/simulate.jl:60
 [5] pmf_lbps(::Dict{String,Any}, ::Dict{String,Any}, ::Bool) at /Users/tlienart/Dropbox/bouncy_exp/pmf/pmf_lbps.jl:92
 [6] pmf_lbps(::Dict{String,Any}, ::Dict{String,Any}) at /Users/tlienart/Dropbox/bouncy_exp/pmf/pmf_lbps.jl:3
 [7] include_string(::String, ::String) at ./loading.jl:515
 [8] include_string(::Module, ::String, ::String) at /Users/tlienart/.julia/v0.6/Compat/src/Compat.jl:464
 [9] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/tlienart/.julia/v0.6/IJulia/src/execute_request.jl:154
 [10] eventloop(::ZMQ.Socket) at /Users/tlienart/.julia/v0.6/IJulia/src/eventloop.jl:8
 [11] (::IJulia.##14#17)() at ./task.jl:335
while l

... simulation finished (95.3s)
... LBPS finished (100.1s)


Progress: 100%|█████████████████████████████████████████| Time: 0:00:56


In [35]:
rmse_train = pmf_rmse(rows[train_mask], cols[train_mask], rates[train_mask],  
                        nU, nV, d, xx)
rmse_test  = pmf_rmse(rows[test_mask], cols[test_mask], rates[test_mask],  
                        nU, nV, d, xx)

println("$rmse_train -- $rmse_test")

0.9885777036477144 -- 1.0027114754343391


In [91]:
var(length(results["ALL_EVLIST"].evl[i].ts) for i in 1:length(results["ALL_EVLIST"].evl))

0.3489218358973028

In [75]:
samplelocalpath(results["ALL_EVLIST"].evl[3], linspace(0,results["SIM_DETAILS"]["globalclock"],40))

40-element Array{Array{Float64,1},1}:
 [-0.0403848, -0.0484269, 0.0450904, -0.0169327, 0.0129558, -0.0801606, 0.0259553, 0.0366208, -0.021325, -0.0321542]
 [-0.0406837, -0.0118641, 0.129956, -0.0432231, -0.0122626, -0.111194, 0.0209135, 0.0439405, -0.077655, -0.0375269] 
 [-0.0409826, 0.0246987, 0.214822, -0.0695135, -0.037481, -0.142228, 0.0158717, 0.0512602, -0.133985, -0.0428996]   
 [-0.0412814, 0.0612615, 0.299688, -0.0958039, -0.0626994, -0.173262, 0.0108299, 0.0585798, -0.190315, -0.0482722]  
 [-0.0415803, 0.0978243, 0.384554, -0.122094, -0.0879178, -0.204296, 0.00578812, 0.0658995, -0.246645, -0.0536449]  
 [-0.0418792, 0.134387, 0.46942, -0.148385, -0.113136, -0.23533, 0.000746336, 0.0732192, -0.302975, -0.0590176]     
 [-0.042178, 0.17095, 0.554285, -0.174675, -0.138355, -0.266364, -0.00429545, 0.0805388, -0.359305, -0.0643902]     
 [-0.0424769, 0.207513, 0.639151, -0.200965, -0.163573, -0.297398, -0.00933724, 0.0878585, -0.415635, -0.0697629]   
 [-0.0427758, 0.244075, 0.

In [57]:
samplepath(results["ALL_EVLIST"], )

LoadError: [91mMethodError: no method matching samplepath(::PDMP.AllEventList)[0m
Closest candidates are:
  samplepath([91m::PDMP.Path[39m, [91m::Float64[39m) at /Users/tlienart/.julia/v0.6/PDMP/src/path.jl:88
  samplepath([91m::PDMP.Path[39m, [91m::Union{Array{Float64,1}, Range{Float64}}[39m) at /Users/tlienart/.julia/v0.6/PDMP/src/path.jl:66[39m

In [ ]:
 # -------------------------------------

open("results.dat","a") do f
    l = "$en : $simtime s : $rmse_test\n"
    print(l)
    write(f, l)
end


## HMC territory

In [24]:
include("pmf_ll.jl")
(ll, gll) = pmf_ll(rows[train_mask], cols[train_mask], rates[train_mask], 
                    nU, nV, sR, sU, sV, d)

(loglik, gradloglik)

## HMC runs

In [25]:
include("hmc.jl")

hmc (generic function with 1 method)

In [26]:
ta = time()
samples = hmc(ll, gll, x0; steps=50, burnin=5, stepsize=0.01);
print(time()-ta)

Progress:  98%|████████████████████████████████████████ |  ETA: 0:00:02

108.81666398048401

Progress: 100%|█████████████████████████████████████████| Time: 0:01:47


In [39]:
size(samples[1])

(99920,)

In [27]:
ss = sum(samples)/length(samples);

In [29]:
pmf_rmse(rows[train_mask], cols[train_mask], rates[train_mask],  nU, nV, d, ss)

0.9872230234264999

In [28]:
pmf_rmse(rows[test_mask], cols[test_mask], rates[test_mask],  nU, nV, d, ss)

1.0014518344059005

In [65]:
size(samples[1])

(99920,)

In [44]:
pmf_rmse2(rows[train_mask], cols[train_mask], rates[train_mask],  nU, nV, d, samples)

0.9477468346227618

In [45]:
pmf_rmse2(rows[test_mask], cols[test_mask], rates[test_mask],  nU, nV, d, samples)

1.0014528276800125

BoundsError: [91mBoundsError: attempt to access "sample"
  at index [0][39m

In [36]:
length(ss)

299760

## ALL Results

In [31]:
errmod(mask, x) = pmf_rmse(rows[mask], cols[mask], rates[mask],  nU, nV, d, x)
println("RMSE train SVD : $(errmod(train_mask, xSVD))")
println("RMSE train LBPS: $(errmod(train_mask, xx))")
println("RMSE train HMC : $(errmod(train_mask, ss))")

println("RMSE test  SVD : $(errmod(test_mask, xSVD))")
println("RMSE test  LBPS: $(errmod(test_mask, xx))")
println("RMSE test  HMC : $(errmod(test_mask, ss))")


RMSE train SVD : 0.9886040071534168
RMSE train LBPS: 0.9885669934683817
RMSE train HMC : 0.9872230234264999
RMSE test  SVD : 1.0027202112661955
RMSE test  LBPS: 1.0026865888012215
RMSE test  HMC : 1.0014518344059005
